### Continuous anaerobic digestion

Let's use the same description we had for the first part of chapter 6:



In [25]:
import numpy
import pandas
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

S1 = numpy.matrix(pandas.read_excel('matdig1.xlsx'))
S2 = numpy.matrix(pandas.read_excel('matdig2.xlsx'))

In [18]:
mumax=0.075 #1/day
thetamax=1/1000*(12+1.7+16*0.55+14*0.04)*24
muDmax=0.02
MMs=12+S1[1,0]+S1[2,0]*16+S1[3,0]*14
Km=1.5/MMs

In [19]:
def response_fun(C):
    
    Cs, Cx, Nch4, Nco2, Cnh3, Cr = C[0],C[1],C[2],C[3],C[4],C[5]
    
    mu= mumax*Cs/(Km+Cs)
    muD=muDmax*Cs/(Km+Cs)
    theta=thetamax*Cs/(Km+Cs)

    C1 = numpy.matrix([[0,0,0,0,mu,0,theta]]).T  #Growth rate specified
    C2 = numpy.matrix([[0,0,0,0,muD]]).T   #Death rate specified

    r1 = numpy.linalg.solve(S1,C1)
    r2 = numpy.linalg.solve(S2,C2)
    
    return [r1[0,0],           # rs
            r1[1,0]+r2[0,0],   # rx
            r1[2,0]+r2[2,0],   # rch4
            r1[3,0]+r2[3,0],   #rco2
            r1[5,0]+r2[1,0],   #rnh3
            r1[6,0]]           #rr   


\begin{align*}
\frac{dC_i}{dt} &= D(C^F_i - C_i) + r_i C_X \\
\end{align*}


In [20]:
V=2 #L

In [21]:
def ferm_eq(C):
    
    Cs, Cx, Cch4, Cco2, Cnh3, Cr = C[0],C[1],C[2],C[3],C[4],C[5]  # order of components very important 
    
    r=response_fun([Cs, Cx, Nch4, Nco2, Cnh3, Cr])      # this is the 'fetching' from the response function
    
    dcsdt=D*(Cf[0]-C[0]) +  r[0]*Cx
    dcxdt=D*(Cf[1]-C[1]) +  r[1]*Cx
    dNch4dt=r[2]*Cx*V
    dNco2dt=r[3]*Cx*V
    dcndt=D*(Cf[4]-C[4]) +  r[4]*Cx
    dcrdt=D*(Cf[5]-C[5]) +  r[5]*Cx
    
    return [dcsdt,
            dcxdt,
            dNch4dt,
            dNco2dt,
            dcndt,
            dcrdt] 

In [22]:
Co=[40/MMs,0,0,0,0,0]

In [24]:
tmax = 100                               # time to integrate to  
tspan=numpy.linspace(0,tmax,200)            # define the timespan of the integration and increments where values will be reported
tper = (0,tmax)                          #defining the starting and end time for the solve_IVP function

CorNi = solve_ivp(ferm_eq, tper, Co, dense_output = True, method = "LSODA")    # integration via odeint that calls apon response_fun

Cs, Cx, Nch4, Nco2, Cnh3,Cr = CorNi.sol(tspan)

plt.figure(3)
plt.plot(tspan, Cx, color='blue', label='Cx')
plt.plot(tspan, Cs, color='green', label='Cs')
plt.plot(tspan, Cnh3, color='red', label='Cnh3')
plt.legend(loc='best')
plt.ylabel('Concentration Cmol/L') 
plt.xlabel('time (days)') 
plt.show() 

NameError: name 'solve_ivp' is not defined